In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ib_color_naming.src.ib_naming_model import load_model
from ib_color_naming.src. figures import mode_map
ib_model = load_model()
from misc import gaussian_model, contour_maps, get_convexity, get_soft_convexity, get_vertical
cielab = pd.read_csv('data/munsell_chart.txt', sep='\t')[['L*', 'a*', 'b*']].values

def generate_random_speaker(k):
    speaker = np.zeros((330, k))
    for i in range(330):
        idx = np.random.choice(k)
        speaker[i, idx] = 1
    return speaker


2024-07-27 07:35:25,994 [INFO] [ib_naming_model - load_model] loading model from file: ./models/IB_color_naming_model/model.pkl


In [2]:
# load wcs speakers 
df = []
with open('data/wcs_encoders.pkl', 'rb') as f:
    wcs_speakers = pickle.load(f)

with open('iterated_learning_exp_cogsci/random_model/encoders.pkl', 'rb') as f:
    random_model = pickle.load(f)

hue_speakers = get_vertical(10)
    # Soft convexity not used in the paper, only hard convexity.
    # Definition didn't make much sense in the end (just some sort of conditional probability?). 
    # Keeping it in case it turns out to be useful for something else.
    # The Hard convexity is the same convexity measure used by Steinert-Threlkeld, J Szymanik 2020. 

for speaker in wcs_speakers:
    hard_conv = get_convexity(speaker)
    soft_conv = get_soft_convexity(speaker, ib_model.pM.flatten())
    results = {'Type': 'WCS' ,'Hard': hard_conv, 'Soft': soft_conv}
    df.append(results)

for speaker in hue_speakers:
    hard_conv = get_convexity(speaker)
    soft_conv = get_soft_convexity(speaker, ib_model.pM.flatten())
    results = {'Type': 'Hue' ,'Hard': hard_conv, 'Soft': soft_conv}
    df.append(results)

for speaker in random_model:
    hard_conv = get_convexity(speaker)
    soft_conv = get_soft_convexity(speaker, ib_model.pM.flatten())
    results = {'Type': 'Random' ,'Hard': hard_conv, 'Soft': soft_conv}
    if hard_conv < 1:
        print(hard_conv)
    df.append(results)



df = pd.DataFrame(df)

In [3]:
print(f"WCS: Hard Conv: {df[df['Type'] == 'WCS']['Hard'].mean()} +/- {df[df['Type'] == 'WCS']['Hard'].std()}")
print(f"Random: Hard Conv: {df[df['Type'] == 'Random']['Hard'].mean()} +/- {df[df['Type'] == 'Random']['Hard'].std()}")
print(f"Hue: Hard Conv: {df[df['Type'] == 'Hue']['Hard'].mean()} +/- {df[df['Type'] == 'Hue']['Hard'].std()}")

# WCS results replicates the ideas of Jäger and Gärdenfors but with a different measure of convexity.
# Value of 1 means perfect convexity, 0 means no convexity.

WCS: Hard Conv: 0.9400272727272728 +/- 0.040150738700325525
Random: Hard Conv: 1.0 +/- 0.0
Hue: Hard Conv: 1.0 +/- 0.0
